In [164]:
import pandas as pd
import sklearn
import re
import nltk
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import string
from gensim import corpora
from gensim.models import LsiModel
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.coherencemodel import CoherenceModel
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [165]:
book_data = pd.read_csv("books.csv")
user_data = pd.read_csv("users.csv")

In [168]:
book_data.keys()

Index(['_id', 'id', 'title', 'isbn', 'isbn13', 'asin', 'kindle_asin',
       'marketplace_id', 'country_code', 'publication_year',
       'publication_month', 'publication_day', 'publisher', 'language_code',
       'is_ebook', 'description', 'work', 'average_rating', 'num_pages',
       'format', 'edition_information', 'ratings_count', 'text_reviews_count',
       'authors', 'popular_shelves', 'book_links', 'series_works',
       'similar_books', 'public_document'],
      dtype='object')

In [169]:
book_data = book_data.drop(['publication_day','isbn','isbn13','publication_month','publication_day','is_ebook','work','average_rating','num_pages','format','edition_information','ratings_count','book_links','series_works','similar_books'],axis = 1)


In [171]:
book_data = book_data.drop(['asin','kindle_asin','marketplace_id','country_code','text_reviews_count','public_document'], axis = 1)

In [172]:
book_data.keys()

Index(['_id', 'id', 'title', 'publication_year', 'publisher', 'language_code',
       'description', 'authors', 'popular_shelves'],
      dtype='object')

In [173]:
book_data = book_data.drop(['_id','id'], axis = 1)

In [177]:
book_data.head()

,title,publication_year,publisher,language_code,description,authors,popular_shelves
0,Harry Potter and the Half-Blood Prince (Harry ...,2006.0,Scholastic Inc.,eng,When Harry Potter and the Half-Blood Prince op...,"{'author': [{'id': '1077326', 'name': 'J.K. Ro...","{'shelf': [{'@name': 'to-read', '@count': '253..."
1,Harry Potter and the Order of the Phoenix (Har...,2004.0,Scholastic Inc.,eng,There is a door at the end of a silent corrido...,"{'author': [{'id': '1077326', 'name': 'J.K. Ro...","{'shelf': [{'@name': 'fantasy', '@count': '439..."
2,Harry Potter and the Sorcerer's Stone (Harry P...,1997.0,Scholastic Inc,eng,Harry Potter's life is miserable. His parents ...,"{'author': [{'id': '1077326', 'name': 'J.K. Ro...","{'shelf': [{'@name': 'to-read', '@count': '795..."
3,Harry Potter and the Chamber of Secrets (Harry...,2003.0,Scholastic,eng,The Dursleys were so mean and hideous that sum...,"{'author': {'id': '1077326', 'name': 'J.K. Row...","{'shelf': [{'@name': 'to-read', '@count': '208..."
4,Harry Potter and the Prisoner of Azkaban (Harr...,2004.0,Scholastic Inc.,eng,Harry Potter's third year at Hogwarts is full ...,"{'author': [{'id': '1077326', 'name': 'J.K. Ro...","{'shelf': [{'@name': 'to-read', '@count': '217..."


In [178]:
for i in range (book_data.shape[0]):
        book_data.iloc[i,6] = re.sub(r"[^a-zA-Z]+", ' ', book_data.iloc[i,6])

In [179]:
word = ['name','count','shelf']
for i in range(book_data.shape[0]):
    word_list = book_data.iloc[i,6].split();
    book_data.iloc[i,6] = ' '.join([i for i in word_list if i not in word])


In [180]:
for i in range (book_data.shape[0]):
        book_data.iloc[i,5] = re.sub(r"[^a-zA-Z]+", ' ', book_data.iloc[i,5])

In [181]:
word = ['name','count','shelf']
for i in range(book_data.shape[0]):
    word_list = book_data.iloc[i,6].split();
    book_data.iloc[i,6] = ' '.join([i for i in word_list if i not in word])


In [184]:
def pre_process(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", " ")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", " "))
        else:
            return ''

In [185]:
columns = ['description','authors','publisher','popular_shelves']

for f in columns:
    book_data[f] = book_data[f].apply(pre_process)

In [186]:
def combine(x):
    return  (x['description']) + ' ' + (x['authors']) + ' ' + (x['publisher']) + ' ' + (x['popular_shelves'])

In [188]:
book_data['train'] = book_data.apply(combine,axis = 1)

In [190]:
word_stopped = TfidfVectorizer(stop_words='english')

book_data['train'] = book_data['train'].fillna('')

matrix = word_stopped.fit_transform(book_data['train'])


In [192]:
co_sim = linear_kernel(matrix,matrix)

In [194]:
book_data = book_data.reset_index()

In [195]:
indexing = pd.Series(book_data.index, index=book_data['title']).drop_duplicates()

In [ ]:
indexing = pd.Series(book_data.index, index=book_data['title','publisher']).drop_duplicates()

In [223]:
resultss = []

In [228]:
def get_recommendations(title,cosine_sim1 = co_sim):
    index = indexing[title]
    similarity = list(enumerate(cosine_sim1[index]))
    
    similarity = sorted(similarity,key=lambda i: i[1], reverse = True)
    
    similarity = similarity[1:20]
    
    movie_index = [j[0] for j in similarity ]
    resultss = book_data['title'].iloc[movie_index]
    return resultss

In [229]:
book_data['title'][323]

'The Door Into Summer'

In [230]:
get_recommendations('The Door Into Summer')

328                                  Time Enough for Love
329                             To Sail Beyond the Sunset
331        Time for the Stars (Heinlein's Juveniles, #10)
330                              Job: A Comedy of Justice
326              Starman Jones (Heinlein's Juveniles, #7)
325                            Stranger in a Strange Land
327                         The Notebooks of Lazarus Long
105          Dune Trilogy Box Set (Dune Chronicles, #1-3)
324                          The Moon is a Harsh Mistress
336     Life, the Universe and Everything (Hitchhiker'...
103                 Children of Dune (Dune Chronicles #3)
1945            Whipping Star (ConSentiency Universe, #1)
97                      Dune Messiah (Dune Chronicles #2)
94              God Emperor of Dune (Dune Chronicles, #4)
322     Off the Main Sequence: The Other Science Ficti...
1944    The Dosadi Experiment (ConSentiency Universe, #2)
15      The Hitchhiker's Guide to the Galaxy (Hitchhik...
108           

In [126]:
get_recommendations("Darwin's Dangerous Idea: Evolution and the Meanings of Life")

2003                                      Freedom Evolves
2005                               The Intentional Stance
2010    Elbow Room: The Varieties of Free Will Worth W...
2001                              Consciousness Explained
2002    Sweet Dreams: Philosophical Obstacles to a Sci...
1999    Breaking the Spell: Religion as a Natural Phen...
2031     Stephen Hawking's Universe: The Cosmos Explained
2004    Brainstorms: Philosophical Essays on Mind and ...
2006    Kinds of Minds: Towards an Understanding of Co...
2008    Kinds of Minds: The Origins Of Consciousness (...
Name: title, dtype: object

In [162]:
get_recommendations("Darwin's Dangerous Idea: Evolution and the Meanings of Life")

2014    Index To The Dennett Quartet: A Boxed Set Of B...
2001                              Consciousness Explained
2178    Daily Reflections For Highly Effective People:...
2005                               The Intentional Stance
2175    The 7 Habits of Highly Effective People Person...
2003                                      Freedom Evolves
1830    High Exposure: An Enduring Passion for Everest...
1499                                     Oedipus the King
2408                                     The Time Machine
1565                                        Twelfth Night
Name: title, dtype: object

In [200]:
get_recommendations("Darwin's Dangerous Idea: Evolution and the Meanings of Life")

2003                                      Freedom Evolves
1928    The Third Chimpanzee: The Evolution & Future o...
2001                              Consciousness Explained
1927    The Rise And Fall Of The Third Chimpanzee: how...
1926    Why Is Sex Fun? The Evolution of Human Sexuali...
2080    In The Blink Of An Eye: How Vision Sparked The...
2010    Elbow Room: The Varieties of Free Will Worth W...
2027                           The Universe in a Nutshell
1999    Breaking the Spell: Religion as a Natural Phen...
2008    Kinds of Minds: The Origins Of Consciousness (...
Name: title, dtype: object

In [203]:
get_recommendations('Storage Area Network Essentials: A Complete Guide to Understanding and Implementing Sans')

512                  Storage Network Performance Analysis
511                     Storage Area Networks For Dummies
514                      Taunton's Home Storage Idea Book
510     USB Mass Storage: Designing and Programming De...
692                               Jazz Guitar Chord Solos
671     America (The Book): A Citizen's Guide to Democ...
513                                  Complete Home Storge
1746                         Correctional Case Management
1747    Alternative Sentencing: Electronically Monitor...
2225                 The Emergence of Sociological Theory
Name: title, dtype: object

In [208]:
get_recommendations('Letterhead and Logo Design 9')

614     Logo Design Workbook: A Hands-On Guide to Crea...
2684       History of Graphic Design--Instructor's Manual
2701       Type and Image: The Language of Graphic Design
2681                     Meggs' History of Graphic Design
615     LOGO Lounge: 2,000 International Identities by...
2683     Graphic Design: A Concise History (World of Art)
655                                      The End of Print
617                 Los Logos: A Selected LOGO Collection
2703           Typographic Design: Form and Communication
1802    The Elements of User Experience: User-Centered...
Name: title, dtype: object

In [213]:
get_recommendations('What to Expect the First Year (What to Expect)')

2430    The Baby Name Wizard: A Magical Method for Fin...
2531    Great Expectations: Your All-In-One Resource f...
2148                   Your Baby's First Year for Dummies
2436                               50,001 Best Baby Names
2339    The Three-Martini Playdate: A Practical Guide ...
2146                             Baby Signing For Dummies
2648    My Boys Can Swim!: The Official Guy's Guide to...
896     Los 5 Lenguajes Del Amor De Los Ninos / The Fi...
904                   The Five Love Languages of Children
2435                                Cool Names for Babies
Name: title, dtype: object

In [217]:
get_recommendations('The Fellowship of the Ring (The Lord of the Rings, #1)')

31      The Lord of the Rings (The Lord of the Rings, ...
33      The Lord of the Rings (The Lord of the Rings, ...
36                          The Lord of the Rings Box Set
30      The Lord of the Rings (The Lord of the Rings, ...
28      J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...
2251    The Lord of the Rings- 3 volumes set (The Lord...
37                                  The Lord of the Rings
29      The Lord of the Rings (The Lord of the Rings, ...
2253                                  Farmer Giles of Ham
2250              The Languages of Tolkien's Middle-Earth
2249    The History of the Lord of the Rings (The Hist...
105          Dune Trilogy Box Set (Dune Chronicles, #1-3)
2812                                   The Worm Ouroboros
2811                                   The Worm Ouroboros
336     Life, the Universe and Everything (Hitchhiker'...
15      The Hitchhiker's Guide to the Galaxy (Hitchhik...
35       The Lord of the Rings: Complete Visual Companion
2247          

In [227]:
    get_recommendations('eBay for Dummies')

128                 Starting an eBay Business for Dummies
131     eBay Business All-in-One Desk Reference for Du...
130                ebay Timesaving Techniques for Dummies
123     How to Buy, Sell & Profit on eBay: Kick-Start ...
127     Titanium eBay: A Tactical Guide to Becoming a ...
129                eBay: Top 100 Simplified Tips & Tricks
126     What to Sell on ebay and Where to Get It: The ...
125     eBay PowerSeller Secrets: Insider Tips from eB...
2487    The Long Tail:  Why The Future Is Selling Less...
589                   The Automatic Millionaire Homeowner
637                                   Sailing for Dummies
1004                           The Richest Man in Babylon
2529    The Innovator's Solution: Creating and Sustain...
2145                                      Sex For Dummies
955     The Millionaire Next Door: The Surprising Secr...
950     The Millionaire Next Door: The Surprising Secr...
2580    The Wealthy Barber: The Common Sense Guide to ...
2581    The We